#Goal- Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model



In [42]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [43]:
## Split the data into training,validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [44]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8])

In [45]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [46]:
np.mean(train_x,axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

In [47]:
train_x.shape[1]

11

In [48]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        mlflow.tensorflow.log_model(model,"model",signature=signature)
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [49]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [50]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [54]:
mlflow.set_experiment("wine-quality_detection")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/08/07 16:11:18 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality_detection' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025/08/07 16:11:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 193ms/step - loss: 36.3964 - root_mean_squared_error: 6.0329
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.1790 - root_mean_squared_error: 6.0149 - val_loss: 35.9397 - val_root_mean_squared_error: 5.9950

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 32.6652 - root_mean_squared_error: 5.7154
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.2007 - root_mean_squared_error: 5.9330 - val_loss: 34.9647 - val_root_mean_squared_error: 5.9131

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 35.6080 - root_mean_squared_error: 5.9672
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - loss: 34.2497 - root_mean_squared_error: 5.8523 - val_loss: 34.0227 - val_root_mean_squared_error: 5.8329

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 34.4046 - root_mean_squared_error: 5.8655
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025/08/07 16:11:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 143ms/step - loss: 39.2645 - root_mean_squared_error: 6.2661
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.8788 - root_mean_squared_error: 3.5887 - val_loss: 2.9334 - val_root_mean_squared_error: 1.7127

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9136 - root_mean_squared_error: 1.3833
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.8754 - root_mean_squared_error: 1.3694 - val_loss: 1.5677 - val_root_mean_squared_error: 1.2521

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.5424 - root_mean_squared_error: 1.2419
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.3922 - root_mean_squared_error: 1.1799 - val_loss: 1.3567 - val_root_mean_squared_error: 1.1648

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.33

2025/08/07 16:11:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:34<00:00,  8.57s/trial, best loss: 0.7335559725761414]

2025/08/07 16:11:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.044625503053587486), 'momentum': np.float64(0.7779300635549113)}
Best eval rmse: 0.7335559725761414


In [37]:
mlflow.register_model(
    "runs:/3a6c037078b64c3dac954d1189637278/model",
    "BestModel"
)

Registered model 'BestModel' already exists. Creating a new version of this model...
2025/08/07 15:57:47 WARNING mlflow.tracking._model_registry.fluent: Run with id 3a6c037078b64c3dac954d1189637278 has no artifacts at artifact path 'model', registering model based on models:/m-a9164cd2d91e46f38752b9379eaa1d23 instead
Created version '1' of model 'BestModel'.


<ModelVersion: aliases=[], creation_timestamp=1754553467235, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1754553467235, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-a9164cd2d91e46f38752b9379eaa1d23', run_id='3a6c037078b64c3dac954d1189637278', step=0, timestamp=1754552909557, value=0.7369639277458191>], model_id='m-a9164cd2d91e46f38752b9379eaa1d23', name='BestModel', params={'lr': '0.006544211542246165', 'momentum': '0.9324151459059883'}, run_id='3a6c037078b64c3dac954d1189637278', run_link=None, source='models:/m-a9164cd2d91e46f38752b9379eaa1d23', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [55]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = "runs:/93a53dd2cb9941e9b2c2db9c543cd5ff/model"

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step


array([[6.0006075],
       [7.024006 ],
       [6.794571 ],
       ...,
       [6.517021 ],
       [6.889268 ],
       [5.9836426]], dtype=float32)

In [56]:
# Load model as a PyFuncModel.
model_uri = "runs:/93a53dd2cb9941e9b2c2db9c543cd5ff/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step 


array([[6.0006075],
       [7.024006 ],
       [6.794571 ],
       ...,
       [6.517021 ],
       [6.889268 ],
       [5.9836426]], dtype=float32)

In [57]:
## Register in the model registry
mlflow.register_model(model_uri,"BestModel_wine-quality")

Successfully registered model 'BestModel_wine-quality'.
2025/08/07 16:15:11 WARNING mlflow.tracking._model_registry.fluent: Run with id 93a53dd2cb9941e9b2c2db9c543cd5ff has no artifacts at artifact path 'model', registering model based on models:/m-3b30fd5a9bee4d8eb8e4e9a51b22ab16 instead
Created version '1' of model 'BestModel_wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1754554511178, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1754554511178, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-3b30fd5a9bee4d8eb8e4e9a51b22ab16', run_id='93a53dd2cb9941e9b2c2db9c543cd5ff', step=0, timestamp=1754554313099, value=0.7335559725761414>], model_id='m-3b30fd5a9bee4d8eb8e4e9a51b22ab16', name='BestModel_wine-quality', params={'lr': '0.044625503053587486', 'momentum': '0.7779300635549113'}, run_id='93a53dd2cb9941e9b2c2db9c543cd5ff', run_link=None, source='models:/m-3b30fd5a9bee4d8eb8e4e9a51b22ab16', status='READY', status_message=None, tags={}, user_id=None, version=1>